# **Machine Learning Skin Lesion Malignant or Benign using CNN**

# **Disclaimer**

* Production Readiness and Model Quality: The primary objective of this project is to demonstrate a viable technical pipeline for model conversion, specifically from a JIT-compiled PyTorch model to a TFLite format for integration with Android using AI Edge Gallery and other platforms like Ollama/Gemma3N on an offline laptop. This model is not production-ready. It has not been subjected to rigorous performance, accuracy, or bias testing. The code serves as a proof-of-concept for the conversion process, and any further development would require extensive model improvement and validation before real-world deployment.

* Not a Medical Tool; No Medical Advice: The output from this model, whether used in an Android app or integrated with an LLM like Gemma3N, is not a substitute for professional medical advice, diagnosis, or treatment. This model is an experimental tool for pattern recognition and should never be used as a replacement for consulting a qualified healthcare professional.

* Framework-Specifics: This project outlines a successful conversion path, but the process may not be universally applicable to all PyTorch models. The success of the ONNX export and subsequent conversion depends heavily on the specific layers and operations used in the original PyTorch model. Certain custom or less common operations might not have direct ONNX or TensorFlow equivalents, leading to conversion failures.

* Performance and Overhead: The use of tf.py_function to wrap the ONNX model is a powerful workaround, but it introduces overhead. The TFLite interpreter will need to perform a context switch to execute the native Python code via ONNX Runtime for every inference call. This can negate some of the performance benefits of using TFLite, especially on low-power devices. The final TFLite model is not a fully "native" TensorFlow Lite model.

* TFLite Custom Operator Support: The resulting TFLite model relies on a custom operator to run the ONNX inference. The ability to run this custom TFLite model on a target device is dependent on the TFLite interpreter's support for custom ops. In many production scenarios, you would need to write a C++ implementation of the custom operator to make it portable and efficient.

* Assumptions and Placeholders: The code contains assumptions, such as the file path /kaggle/input/... and a placeholder for the output shape within tf.py_function. These must be carefully updated and validated for any specific use case. The project assumes a JIT-compiled PyTorch model, and the process for non-JIT models might be slightly different.

* Lack of Quantization: The pipeline does not include any model quantization, which is a critical step for deploying models on resource-constrained devices to reduce size and improve latency.

* This is utilized for working on an output for skin cancer with the existing dataset in Kaggle. If there is more data then definitely all of the classifications would yield a better output.  Data and Generalization: The effectiveness of the final TFLite model is entirely dependent on the quality, diversity, and quantity of the data used to train the original PyTorch model. If the training data for the "skin cancer model" did not include a wide range of skin types, cancer types, or lighting conditions, the model may perform poorly when deployed in the real world. The conversion process itself does not address or mitigate any biases or deficiencies in the original training data.

# **Model Training Data**


https://www.kaggle.com/datasets/fanconic/skin-cancer-malignant-vs-benign

# **Model Training Code**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import numpy as np
import os
import matplotlib.pyplot as plt

# ⚙️ Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [2]:
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        # Adaptive to any input size
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 14 * 14, 256),  # Adjust depending on input size
            nn.ReLU(),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [3]:
%pwd
%ls -al /kaggle/input/skin-cancer-malignant-vs-benign/

total 4
drwxr-xr-x 5 nobody nogroup    0 Jul 11 18:44 ./
drwxr-xr-x 3 root   root    4096 Aug  1 17:02 ../
drwxr-xr-x 4 nobody nogroup    0 Jul 11 18:44 data/
drwxr-xr-x 4 nobody nogroup    0 Jul 11 18:44 test/
drwxr-xr-x 4 nobody nogroup    0 Jul 11 18:44 train/


In [4]:
%pwd
%ls -al /kaggle/input/skin-cancer-malignant-vs-benign/data

total 0
drwxr-xr-x 4 nobody nogroup 0 Jul 11 18:44 ./
drwxr-xr-x 5 nobody nogroup 0 Jul 11 18:44 ../
drwxr-xr-x 4 nobody nogroup 0 Jul 11 18:44 test/
drwxr-xr-x 4 nobody nogroup 0 Jul 11 18:44 train/


In [5]:
# 🧼 Image preprocessing
img_size = 112
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 📂 Dataset setup
train_path = '/kaggle/input/skin-cancer-malignant-vs-benign/train'
val_path = '/kaggle/input/skin-cancer-malignant-vs-benign/test'

train_data = datasets.ImageFolder(train_path, transform=transform)
val_data = datasets.ImageFolder(val_path, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)

In [6]:
# 📦 Model, Loss, Optimizer
model = CustomCNN().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [7]:
def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        # 📊 Validation
        model.eval()
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.float().unsqueeze(1).to(device)
                outputs = model(inputs)
                preds = (torch.sigmoid(outputs) > 0.5).float()
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        acc = accuracy_score(all_labels, all_preds)

        print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_train_loss:.4f} | Val Accuracy: {acc:.4f}")

train_model(model, train_loader, val_loader, epochs=10)

Epoch 1/10 | Loss: 0.4427 | Val Accuracy: 0.8242
Epoch 2/10 | Loss: 0.3694 | Val Accuracy: 0.8500
Epoch 3/10 | Loss: 0.3323 | Val Accuracy: 0.8591
Epoch 4/10 | Loss: 0.2939 | Val Accuracy: 0.8515
Epoch 5/10 | Loss: 0.2744 | Val Accuracy: 0.8197
Epoch 6/10 | Loss: 0.2370 | Val Accuracy: 0.8530
Epoch 7/10 | Loss: 0.2227 | Val Accuracy: 0.8636
Epoch 8/10 | Loss: 0.1898 | Val Accuracy: 0.8500
Epoch 9/10 | Loss: 0.1820 | Val Accuracy: 0.8576
Epoch 10/10 | Loss: 0.1512 | Val Accuracy: 0.8682


# **Model Results**

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(model, val_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(inputs)
            preds = (torch.sigmoid(outputs) > 0.5).float()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Convert to numpy arrays
    y_true = np.array(all_labels)
    y_pred = np.array(all_preds)

    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    print("\n📊 Final Evaluation Metrics:")
    print(f"✅ Accuracy : {acc:.4f}")
    print(f"🎯 Precision: {precision:.4f}")
    print(f"🔁 Recall   : {recall:.4f}")
    print(f"📈 F1 Score : {f1:.4f}")

    return acc, precision, recall, f1

# Call this after training
evaluate_model(model, val_loader)


📊 Final Evaluation Metrics:
✅ Accuracy : 0.8682
🎯 Precision: 0.8257
🔁 Recall   : 0.9000
📈 F1 Score : 0.8612


(0.8681818181818182, 0.8256880733944955, 0.9, 0.861244019138756)

In [9]:
from PIL import Image

def predict_image(model, image_path, transform, threshold=0.5):
    """
    Predicts the class of a single image using the trained model.
    """
    model.eval()

    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        output = model(input_tensor)
        prob = torch.sigmoid(output).item()
        prediction = 1 if prob > threshold else 0

    print(f"\n🖼️ Prediction Result for '{image_path}':")
    print(f"🔢 Raw Score (Sigmoid): {prob:.4f}")
    print(f"🧠 Predicted Class     : {'Malignant' if prediction == 1 else 'Benign'}")

    return prediction, prob

In [10]:
# 🔍 Replace this with your test image path
test_image_path = "/kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1.jpg"
 

# 🔮 Predict
predict_image(model, test_image_path, transform)


🖼️ Prediction Result for '/kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1.jpg':
🔢 Raw Score (Sigmoid): 0.3693
🧠 Predicted Class     : Benign


(0, 0.36933571100234985)

In [11]:
# 🔍 Replace this with your test image path
test_image_path = "/kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1.jpg"
 

# 🔮 Predict
predict_image(model, test_image_path, transform)


🖼️ Prediction Result for '/kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1.jpg':
🔢 Raw Score (Sigmoid): 0.9931
🧠 Predicted Class     : Malignant


(1, 0.9931436777114868)

# **Save the model**

In [12]:
# 🔐 Save PyTorch model
torch.save(model.state_dict(), "custom_cnn.pth")

# 🧪 Convert to TorchScript for mobile
example_input = torch.randn(1, 3, img_size, img_size).to(device)
traced_model = torch.jit.trace(model, example_input)
traced_model.save("custom_cnn_mobile.pt")

In [13]:
%pwd
%ls -al

total 50980
drwxr-xr-x 3 root root     4096 Aug  1 17:29 ./
drwxr-xr-x 5 root root     4096 Aug  1 17:02 ../
-rw-r--r-- 1 root root 26110150 Aug  1 17:29 custom_cnn_mobile.pt
-rw-r--r-- 1 root root 26076962 Aug  1 17:29 custom_cnn.pth
drwxr-xr-x 2 root root     4096 Aug  1 17:02 .virtual_documents/


# **Outputting model to PyTorch PT format and also TFLite for Mobile and Laptop integration with LLM**

This is on my laptop when I ran the script below manually via command prompt on a Windows 11 (this was painful to copy and modify but I hope you appreciate this):
>
> pip list
>
> Package                  Version
------------------------ -----------
> 
> absl-py                  2.3.1
> 
> aiohappyeyeballs         2.6.1
> 
> aiohttp                  3.12.15
> 
> aiosignal                1.4.0
> 
> annotated-types          0.7.0
> 
> anyio                    4.9.0
> 
> astunparse               1.6.3
> 
> attrs                    25.3.0
> 
> certifi                  2025.7.14
> 
> charset-normalizer       3.4.2
> 
> click                    8.2.1
> 
> colorama                 0.4.6
> 
> coloredlogs              15.0.1
> 
> dataclasses-json         0.6.7
> 
> fastapi                  0.116.1
> 
> filelock                 3.18.0
> 
> flatbuffers              25.2.10
> 
> frozenlist               1.7.0
> 
> fsspec                   2025.7.0
> 
> gast                     0.6.0
> 
> google-pasta             0.2.0
> 
> greenlet                 3.2.3
> 
> grpcio                   1.74.0
> 
> h11                      0.16.0
> 
> h5py                     3.14.0
> 
> httpcore                 1.0.9
> 
> httpx                    0.28.1
> 
> httpx-sse                0.4.1
> 
> humanfriendly            10.0
> 
> idna                     3.10
> 
> intel-cmplr-lic-rt       2025.2.0
> 
> intel-opencl-rt          2025.2.0
> 
> Jinja2                   3.1.6
> 
> joblib                   1.5.1
> 
> jsonpatch                1.33
> 
> jsonpointer              3.0.0
> 
> keras                    3.11.1
> 
> langchain                0.3.27
> 
> langchain-community      0.3.27
> 
> langchain-core           0.3.72
> 
> langchain-text-splitters 0.3.9
> 
> langsmith                0.4.8
> 
> libclang                 18.1.1
> 
> Markdown                 3.8.2
> 
> markdown-it-py           3.0.0
> 
> MarkupSafe               3.0.2
> 
> marshmallow              3.26.1
> 
> mdurl                    0.1.2
> 
> ml_dtypes                0.5.3
> 
> mpmath                   1.3.0
> 
> multidict                6.6.3
> 
> mypy_extensions          1.1.0
> 
> namex                    0.1.0
> 
> networkx                 3.5
> 
> numpy                    2.3.2
> 
> ollama                   0.5.1
> 
> onnx                     1.18.0
> 
> onnx-tf                  1.6.0
> 
> onnxruntime              1.22.1
> 
> opt_einsum               3.4.0
> 
> optree                   0.17.0
> 
> orjson                   3.11.1
> 
> packaging                25.0
> 
> pandas                   2.3.1
> 
> pillow                   11.3.0
> 
> pip                      25.1.1
> 
> propcache                0.3.2
> 
> protobuf                 6.31.1
> 
> pydantic                 2.11.7
> 
> pydantic_core            2.33.2
>
> pydantic-settings        2.10.1
> 
> Pygments                 2.19.2
> 
> pyreadline3              3.5.4
> 
> python-dateutil          2.9.0.post0
> 
> python-dotenv            1.1.1
> 
> pytz                     2025.2
> 
> PyYAML                   6.0.2
> 
> requests                 2.32.4
> 
> requests-toolbelt        1.0.0
> 
> rich                     14.1.0
> 
> scikit-learn             1.7.1
> 
> scipy                    1.16.1
> 
> setuptools               80.9.0
> 
> six                      1.17.0
> 
> sniffio                  1.3.1
>
> SQLAlchemy               2.0.42
> 
> starlette                0.47.2
> 
> sympy                    1.14.0
> 
> tbb                      2022.2.0
> 
> tcmlib                   1.4.0
> 
> tenacity                 9.1.2
> 
> tensorboard              2.20.0
> 
> tensorboard-data-server  0.7.2
> 
> tensorflow               2.20.0rc0
> 
> termcolor                3.1.0
> 
> threadpoolctl            3.6.0
> 
> torch                    2.7.1
> 
> torchvision              0.22.1
> 
> typing_extensions        4.14.1
> 
> typing-inspect           0.9.0
> 
> typing-inspection        0.4.1
> 
> tzdata                   2025.2
> 
> urllib3                  2.5.0
> 
> uvicorn                  0.35.0
> 
> Werkzeug                 3.1.3
> 
> wheel                    0.45.1
> 
> wrapt                    1.17.2
> 
> yarl                     1.20.1
> 
> zstandard                0.23.0 


In [4]:
pip install tf2onnx

ERROR: Could not find a version that satisfies the requirement tf2onnx (from versions: none)
ERROR: No matching distribution found for tf2onnx
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install onnxruntime-rocm

In [2]:
!pip install onnx onnx-tf 

ERROR: Could not find a version that satisfies the requirement onnx-tf (from versions: none)
ERROR: No matching distribution found for onnx-tf


In [3]:
!pip install onnxruntime keras

ERROR: Could not find a version that satisfies the requirement onnxruntime (from versions: none)
ERROR: No matching distribution found for onnxruntime


In [ ]:
# You may have to restart the cluster, but keep the .pt file available to run below
# You could also put all of these pip install to the top of the notebook and then run below later.
# I put these here so that you know what packages are needed to convert the Pytorch .pt model to onnx and tflite to integrate with AI Edge Gallery or Python/Ollama

In [ ]:
# Output this in the format for edge devices (Laptop/Ollama/Gemma3 for phase 1 and Android in phase 2)
# I made this run locally on my laptop but it also works on Kaggle. To make this work on your laptop, just download the .pt file into a folder and change the paths.
import torch
import onnx
import onnxruntime as ort
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np

# --- Part 1: Export the PyTorch JIT model to ONNX ---

# Load the JIT-compiled model. Note: The path below is a placeholder and should be updated.
device = torch.device("cpu")
try:
    cnn_model = torch.jit.load("/kaggle/working/custom_cnn_mobile.pt", map_location=device)
    cnn_model.eval()
except FileNotFoundError:
    print("Error: PyTorch model not found. Please verify the path.")
    # Exit or handle the error gracefully
    cnn_model = None

if cnn_model:
    # Create the dummy input tensor with the correct size.
    # The shape should match the expected input of your PyTorch model.
    dummy_input = torch.randn(1, 3, 112, 112, requires_grad=True)

    # Export the JIT model to ONNX format.
    onnx_model_path = "custom_cnn_mobile_model.onnx"
    torch.onnx.export(cnn_model,
                      dummy_input,
                      onnx_model_path,
                      opset_version=11,
                      input_names=['input'],
                      output_names=['output'],
                      dynamic_axes={'input' : {0 : 'batch_size'}})

    print("Step 1: JIT model has been successfully exported to ONNX.")

    # --- Part 2: Convert ONNX to a Keras model and then to TFLite ---

    # Use ONNX Runtime to get the model's output
    sess = ort.InferenceSession(onnx_model_path, providers=['CPUExecutionProvider'])
    output_name = sess.get_outputs()[0].name
    input_name = sess.get_inputs()[0].name
    
    # Create a helper function to run ONNX inference, which will be called by tf.py_function
    def _run_onnx_inference(input_data):
        """Helper function to run the ONNX session with a numpy array."""
        # Ensure the input is a contiguous numpy array, which is often required
        # by ONNX Runtime and can resolve data format issues from tf.py_function.
        outputs = sess.run([output_name], {input_name: np.ascontiguousarray(input_data)})
        return outputs[0]

    # Create a simple Keras model that wraps the ONNX Runtime session.
    class ONNXModelWrapper(keras.Model):
        def __init__(self, onnx_session, output_name, input_name):
            super(ONNXModelWrapper, self).__init__()
            self.output_name = output_name
            self.input_name = input_name

        @tf.function(input_signature=[
            tf.TensorSpec(shape=(None, 3, 112, 112), dtype=tf.float32)
        ])
        def call(self, inputs):
            # Use tf.py_function to wrap the ONNX inference call.
            # We must specify the output shape and dtype to ensure the graph
            # is correctly traced.
            return tf.py_function(
                _run_onnx_inference,
                inp=[inputs],
                Tout=tf.float32,
                # Explicitly set the output shape to allow for proper graph building.
                # The shape can be inferred from the model's output. For this example,
                # we'll assume a known output shape after the first dimension.
                # You might need to adjust this based on your model's output.
                # Example: tf.TensorShape([None, 1000]) for a 1000-class classifier
                # We will return a placeholder for the output shape here
                # since it's difficult to determine without knowing the model.
                # In practice, you would calculate this from the ONNX model's output.
                name="onnx_inference_op"
            )

    # Instantiate the wrapper model.
    onnx_model_wrapper = ONNXModelWrapper(sess, output_name, input_name)
    onnx_model_wrapper.build(input_shape=(None, 3, 112, 112))
    
    # Call the model once with dummy data to build the graph before exporting.
    dummy_input_tf = tf.convert_to_tensor(dummy_input.detach().numpy())
    _ = onnx_model_wrapper(dummy_input_tf)

    # Save the wrapper as a TensorFlow SavedModel.
    tf_saved_model_path = "tf_saved_model_final"
    onnx_model_wrapper.export(tf_saved_model_path)

    print("Step 2: ONNX model successfully wrapped in a Keras model and saved.")

    # --- Part 3: Convert the SavedModel to TFLite, allowing custom ops ---

    tflite_model_path = "custom_cnn_mobile_model.tflite"

    # Convert the SavedModel to a TFLite model using the native converter.
    # The key change here is to set `allow_custom_ops=True`.
    # This instructs the converter to keep `tf.py_function` as a custom operation
    # instead of trying to convert it to a native TFLite op, which caused the error.
    converter = tf.lite.TFLiteConverter.from_saved_model(tf_saved_model_path)
    converter.allow_custom_ops = True
    tflite_model = converter.convert()

    # Save the TFLite model to a file.
    with open(tflite_model_path, "wb") as f:
        f.write(tflite_model)

    print("Step 3: SavedModel successfully converted to TFLite format.")
    print(f"Final TFLite file saved at: {tflite_model_path}")

# **Understanding the output and interpreting the data**# 

Based on the model's output, here is a summary of the performance:

Final Evaluation Metrics:

Accuracy: 0.8682

Precision: 0.8257

Recall: 0.9000

F1 Score: 0.8612

Performance Over Epochs:

The model was trained for 10 epochs.

Loss: The training loss consistently decreased from 0.4427 in the first epoch to 0.1512 in the final epoch.

Validation Accuracy: The validation accuracy fluctuated during training, with the highest value of 0.8682 achieved in the final epoch (10 
th
  epoch). The lowest validation accuracy was 0.8197, observed in the 5 
th
  epoch.
  

Final Evaluation Metrics
These numbers represent how well the model performed on a final, unseen test dataset after training was complete.

Accuracy (0.8682): This means the model made the correct prediction about 86.82% of the time. It's a good general measure, but it can be misleading for imbalanced datasets.

Precision (0.8257): When the model predicted something was positive, it was correct 82.57% of the time. This metric focuses on the quality of the positive predictions and tells you how trustworthy the model's "yes" answers are.

Recall (0.9000): The model successfully identified 90.00% of all actual positive cases. This metric focuses on the model's ability to find all the positive cases and tells you how good the model is at not missing things.

F1 Score (0.8612): This is the harmonic mean of Precision and Recall. It provides a single score that balances both metrics. A high F1 Score indicates that the model is both good at not making false positive predictions (high precision) and good at not missing positive cases (high recall).

Performance Over Epochs
An "epoch" is one complete pass through the entire training dataset. These numbers show the model's learning progress over time.

Loss: This measures the model's error. A decreasing loss (from 0.4427 to 0.1512) is a good sign, as it means the model is getting better at making predictions on the training data.

Validation Accuracy: This measures the model's performance on a separate "validation" dataset that it hasn't seen before. The goal is for this number to increase. The validation accuracy generally improved over the 10 epochs, reaching its highest point (0.8682) in the final epoch. The fluctuations are normal and indicate the model is adjusting its parameters.

Overall Conclusion
The model is performing very well. The consistently decreasing loss shows that it learned effectively from the training data. The final metrics are strong, with a high accuracy and a great balance between precision and recall, as indicated by the high F1 Score. The model is particularly good at identifying positive cases (high recall) and is reasonably accurate when it makes a positive prediction (good precision).

# **Summary of What I Learned**

This project provided a deep and practical understanding of the challenges and solutions involved in modern machine learning model deployment. The key learnings can be summarized as follows:

* Model Interoperability is Not a Given: While standards like ONNX exist to bridge frameworks, the conversion process is not always a simple one-to-one mapping. It often requires creative workarounds and a solid understanding of how each framework represents and executes models.

* The Power of tf.py_function: I learned that tf.py_function is an essential tool in the TensorFlow ecosystem for integrating custom, non-TensorFlow logic into a TensorFlow graph. It's a key strategy for handling complex, framework-specific operations that would otherwise be impossible to convert directly.

* The Role of Intermediate Formats: ONNX is not just a format; it's a critical bridge in the model lifecycle. I learned to appreciate its role in decoupling the training framework (PyTorch) from the inference engine (TensorFlow Lite via ONNX Runtime), providing greater flexibility and choice.

* The Importance of a Well-Defined Pipeline: The project demonstrated the need for a systematic, multi-step pipeline for model conversion. Each step—from exporting to an intermediate format to wrapping with custom logic and finally converting for a specific target—is a distinct phase with its own set of requirements and potential pitfalls.

* Deployment-Oriented Mindset: I learned that model development doesn't end with training. The entire process, from model design to the final conversion pipeline, should be considered with the deployment target in mind. The choice to use allow_custom_ops and the careful handling of shapes and data types are examples of decisions driven by the end goal of TFLite deployment.
